In [1]:
import math
import networkx as nx

In [2]:
G = nx.Graph()
graph = open("NY_graph.txt","r")

# importance priority queue
imp_pq = list()
order = 0

In [3]:

def Connect(G, graph):
    # get number of nodes and edges
    graph_parameters = graph.readline().split()
    vertices_number = int(graph_parameters[0],base=10)
    edges_number = int(graph_parameters[1],base=10)
    print(vertices_number)
    print(edges_number)
    # add nodes to networkx graph
    for i in range(vertices_number):
        G.add_node(i+1, contracted = False,imp=0,level=0,contr_neighbours=0)
    # add edges to networkx graph
    edge = graph.readline()
    while edge:
        edge_parameters = edge.split()
        source_node = int(edge_parameters[0], base=10)
        target_node = int(edge_parameters[1], base=10)
        edge_weight = int(edge_parameters[2], base=10)
        found_exist = False
        for i in G[source_node]:
            # already store the edge with different weight, choose the min weight
            if i == target_node:
                G[source_node][target_node]['weight'] = min(G[source_node][target_node]['weight'],edge_weight)
                found_exist = True
                break
        if not found_exist:
            G.add_edge(source_node,target_node,weight=edge_weight)
        edge = graph.readline()
        # create undirectional graph
        edge = graph.readline()
    

In [4]:
def SetOrder(G, imp_pq, n):
    # initialize imp_pq
    for i in range(n):
        imp_pq.append((GetImportance(G,i+1),i+1))
        # print("settled node %d"%(i+1))
    order = 1
    # for i in range(1000):
    #     print(imp_pq[i+1])
    lazy_update_counter = 0
    contracted_num = 0
    completed_rate = 0
    k = 0
    print("initializing importance queue settled.")
    while len(imp_pq)>0:
        # find current lowest importance node in imp_pq
        curr_node_imp_pair = min(imp_pq, key= lambda pair:pair[0])
        curr_node  = curr_node_imp_pair[1]   
        imp_pq.remove(curr_node_imp_pair)
        # get new importance for current lowest importance node
        new_imp = GetImportance(G,curr_node)
        # print("get new importance for node %d"%curr_node)
        # lazy update
        if((len(imp_pq) == 0) or (new_imp - min(imp_pq,key=lambda pair:pair[0])[0] <= 10) or (lazy_update_counter >= 5)):
            # print("update for node %d"%curr_node)
            lazy_update_counter = 0
            G.nodes[curr_node]['imp'] = order
            order +=1
            # contract node
            G.nodes[curr_node]['contracted'] = True
            ContractNode(G,curr_node,n)
            # print("already contracted node %d"%curr_node)
            contracted_num += 1
            k +=1
            if(k == 1000):
                completed_rate = contracted_num / n
                print(completed_rate)
                k = 0
        else:
            imp_pq.append((new_imp,curr_node))
            lazy_update_counter +=1
            # print("recalculated prority of node %d" %curr_node)
            

In [5]:
def GetImportance(G, x):
    # get number of incident edges of x
    edges_incident = len(G[x])
    # get number of added shortcut when simulate contracting node x
    shortcuts = 0
    seenBefore = list()
    for i in G[x]:
        for j in G[x]:
            pair = sorted((i,j))
            if (i==j or (pair in seenBefore)):continue
            seenBefore.append(pair)
            if((G.nodes[i]['contracted'] == False) and (G.nodes[j]['contracted'] == False)):
                shortcuts +=1
    edge_difference = shortcuts - edges_incident
    return edge_difference + 2*G.nodes[x]['contr_neighbours'] + G.nodes[x]['level']

In [6]:
def ContractNode(G, x, n):
    mx = GetMaxEdge(G, x)
    seenBefore = list()
    for i in G[x]:
        for j in G[x]:
            if ((G.nodes[i]['contracted'] == True) or (G.nodes[j]['contracted'])):
                continue
            pair = sorted((i,j))
            if (i==j or (pair in seenBefore)):continue
            seenBefore.append(pair)
            Check_Witness(G, n, i, x, mx)
            # print("check witness completed")
    # update importance term in incident node
    for i in G[x]:
        G.nodes[i]['contr_neighbours'] +=1
        G.nodes[i]['level'] = max(G.nodes[i]['level'], G.nodes[x]['level'] + 1)
            

In [7]:
def GetMaxEdge(G, x):
    ret = 0
    for i in G[x]:
        for j in G[x]:
            if((i != j) and (G.nodes[i]['contracted'] == False) and (G.nodes[j]['contracted'] == False)):
                ret = max(ret, G[x][i]['weight'] + G[x][j]['weight'])
    return ret

In [8]:
def Check_Witness(G, n, u, x, mx, type=None):
    # dijkstra priority queue for search witness path from u to v, excludes x
    # v is incident edge of x, excludes u
    D_pq = list()
    # initialize D_pq
    D_pq.append((0, u))
    # distance dictionary from u to any node in search tree
    D_dist = dict()
    # initialize D_dist
    D_dist[u] = 0
    # maximum iteration round for dijkstra search
    iter = int(250 * (n - order) / n)
    while((len(D_pq) > 0) and (iter > 0)):
        iter -=1
        curr_dist_pair = min(D_pq, key= lambda pair:pair[0])
        curr_dist = curr_dist_pair[0]
        a = curr_dist_pair[1]
        D_pq.remove(curr_dist_pair)
        if(curr_dist > D_dist[a]):
            continue
        for p in G[a]:
            new_dist = curr_dist + G[a][p]['weight']
            # p must not be x and not be contracted
            if(p != x and (G.nodes[p]['contracted'] == False)):
                # p must not be settled node or distance greater than new_dist
                if((p not in D_dist) or (D_dist[p] > new_dist)):
                    # prune when witness path greater than mx
                    if(p not in D_dist):
                        if new_dist < mx:
                            D_dist[p] = new_dist
                            D_pq.append((new_dist,p))
                    else:
                        if(D_dist[p] < mx):
                            D_dist[p] = new_dist
                            D_pq.append((new_dist,p))
    for v in G[x]:
        # v can not be u and not be contracted
        if ((v!=u) and (G.nodes[v]['contracted'] == False)):
            new_w = G[u][x]['weight'] + G[x][v]['weight']
            # print("%d %d %d"%(u,v,new_w))
            if((v not in D_dist) or (D_dist[v] > new_w)):
                # add shortcut
                # try:
                #     if(u,v) in G.edges:
                #         print("run here: no more add_edge")
                #         continue
                # except:
                G.add_edge(u,v,weight=new_w)
                # print("run here: add_edge:%d %d"%(u,v))
            

In [9]:
def GetDistance(G, s, t):
    # search with bi-dijkstra with ordering rank
    # initializing dijkstra from source node s
    SP_s = dict()
    parent_s = dict()
    unrelaxed_s = list()
    for node in G.nodes():
        SP_s[node] = math.inf
        parent_s[node] = None
        unrelaxed_s.append(node)
    SP_s[s] = 0
    # dijkstra forward
    while unrelaxed_s:
        node = min(unrelaxed_s, key= lambda node:SP_s[node])
        unrelaxed_s.remove(node)
        if SP_s[node] == math.inf:
            break
        # G[node] are the incident edges of node
        for child in G[node]:
            # skip unqualified edges
            if G.nodes[child]['imp'] < G.nodes[node]['imp']:
                continue
            distance = SP_s[node] + G[node][child]['weight']
            # relax edge
            if distance < SP_s[child]:
                SP_s[child] = distance
                parent_s[child] = node
    # initializing dijkstra from target node t
    SP_t = dict()
    parent_t = dict()
    unrelaxed_t = list()
    for node in G.nodes():
        SP_t[node] = math.inf
        parent_t[node] = None
        unrelaxed_t.append(node)
    SP_t[t] = 0

    # dijkstra backward
    while unrelaxed_t:
        node = min(unrelaxed_t, key= lambda node: SP_t[node])
        unrelaxed_t.remove(node)
        if SP_t[node] == math.inf:
            break
        # G[node] are the incident edges of node
        for child in G[node]:
            # skip unqualified edges
            if G.nodes[child]['imp'] < G.nodes[node]['imp']:
                continue
            distance = SP_t[node] + G[node][child]['weight']
            if distance < SP_t[child]:
                SP_t[child] = distance
                parent_t[child] = node
    minimum = math.inf
    merge_node = None
    for i in SP_s:
        if SP_t[i] == math.inf:
            continue
        if SP_t[i] + SP_s[i] < minimum:
            minimum = SP_s[i] + SP_t[i]
            merge_node = i
    return minimum, merge_node, SP_s, SP_t, parent_s, parent_t

In [10]:
# see the route from origin of dijkstra to a given node
def Route_dijkstra(parent, node):
    route = []
    while node != None:
        route.append(node)
        node = parent[node]
    return route[::-1]

In [11]:
def See_full_route(G, s, t):
    minimum, merge_node, SP_s, SP_t, parent_s, parent_t = GetDistance(G, s, t)
    print("shortest distance:\n")
    print(minimum)
    route_from_target = Route_dijkstra(parent_t, merge_node)
    # show route
    print("route from target:\n")
    print(route_from_target)
    route_from_source = Route_dijkstra(parent_s, merge_node)
    # show route
    print("route from source:\n")
    print(route_from_source)
    route = route_from_source + route_from_target[::-1][1:]
    # show route
    print("entire route:\n")
    print(route)
    unvisited = 0
    for s_node, s_dist in SP_s.items():
        for t_node, t_dist in SP_t.items():
            if s_node == t_node and s_dist == t_dist == math.inf:
                unvisited += 1
    print(f"""we have skipped {unvisited} nodes from a graph with {len(G)}, 
    so we have skipped {unvisited/len(G)*100}% of the nodes in our search space.""")

In [12]:
Connect(G, graph)

264346
733846


In [13]:
edges_before = [*G.edges()]

In [14]:
# G.nodes.data()
for i in range(100):
    print(i+1,G.nodes[i+1])

1 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
2 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
3 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
4 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
5 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
6 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
7 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
8 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
9 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
10 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
11 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
12 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
13 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
14 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
15 {'contracted': False, 'imp

In [15]:
SetOrder(G, imp_pq, len(G.nodes))

initializing importance queue settled.
0.003782920868861265
0.00756584173772253
0.011348762606583795
0.01513168347544506
0.018914604344306325
0.02269752521316759
0.026480446082028857
0.03026336695089012
0.034046287819751386
0.03782920868861265
0.04161212955747392
0.04539505042633518
0.049177971295196445
0.052960892164057714
0.05674381303291898
0.06052673390178024
0.0643096547706415
0.06809257563950277
0.07187549650836404
0.0756584173772253
0.07944133824608657
0.08322425911494784
0.0870071799838091
0.09079010085267036
0.09457302172153163
0.09835594259039289
0.10213886345925416
0.10592178432811543
0.10970470519697668
0.11348762606583795
0.11727054693469922
0.12105346780356048
0.12483638867242175
0.128619309541283
0.1324022304101443
0.13618515127900555
0.1399680721478668
0.14375099301672808
0.14753391388558934
0.1513168347544506
0.15509975562331188
0.15888267649217314
0.1626655973610344
0.16644851822989568
0.17023143909875693
0.1740143599676182
0.17779728083647947
0.18158020170534073
0.18

KeyboardInterrupt: 